In [8]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [9]:
# open final data
data = pd.read_csv('../../data/processed/final_data.csv')

# get the list of columns
columns = data.columns
positive_columns = [col for col in columns if '+' in col]
negative_columns = [col for col in columns if '-' in col]
original_columns = [col for col in columns if '+' not in col and '-' not in col]

In [10]:
print(original_columns)

['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count', 'year', 'month', 'day', 'hour', 'dataset', 'dayofyear', 'weekofyear', 'dayofweek']


In [11]:
# use only the training data
model_training_data = data[data['dataset'] == 'train']

# convert the datetime columns to datetime
model_training_data['datetime'] = pd.to_datetime(model_training_data['datetime'])

/var/folders/bn/prwtql5x5sg0mbkxc6508fqm0000gn/T/ipykernel_85437/545332924.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_training_data['datetime'] = pd.to_datetime(model_training_data['datetime'])


In [12]:
# mask the data from day 3 to 15 of each month using 'datetime' column
negative_train_mask = model_training_data['datetime'].apply(lambda x: x.day >= 3 and x.day <= 15)
negative_test_mask = model_training_data['datetime'].apply(lambda x: x.day > 15)

# get the negative training data
negative_train_data = model_training_data[negative_train_mask][original_columns + negative_columns].copy()
negative_test_data = model_training_data[negative_test_mask][original_columns + negative_columns].copy()

# maske the data from day 5 to 17 of each month using 'datetime' column
positive_train_mask = model_training_data['datetime'].apply(lambda x: x.day >= 5 and x.day <= 17)
positive_test_mask = model_training_data['datetime'].apply(lambda x: x.day < 5)

# get the positive training data
positive_train_data = model_training_data[positive_train_mask][original_columns + positive_columns].copy()
positive_test_data = model_training_data[positive_test_mask][original_columns + positive_columns].copy()

# save the data into a dictionary
training_data = {
    'positive': {
        'X': positive_train_data[original_columns],
        'y': positive_train_data[positive_columns]
    },
    'negative': {
        'X': negative_train_data[original_columns],
        'y': negative_train_data[negative_columns]
    }
}

In [13]:
# for each train or test assert there are no NaN values
assert not negative_train_data.isnull().values.any()
assert not negative_test_data.isnull().values.any()
assert not positive_train_data.isnull().values.any()
assert not positive_test_data.isnull().values.any()

In [14]:
def create_pipelines(target_columns, directions, rf=True, mlp=False):
    # create a dictionary to store the pipelines
    pipelines = {}

    for target in target_columns:
        for direction in directions:
                
            if rf:
                # create random forest pipeline for the target and direction
                globals()[f'{direction}_{target}_pipeline_rf'] = Pipeline([
                    ('scaler', StandardScaler()),
                    ('regressor', RandomForestRegressor(n_estimators=100)),
                ])

                # save the pipeline
                pipelines[f'{direction}_{target}_pipeline_rf'] = globals()[f'{direction}_{target}_pipeline_rf']
            
            if mlp:
                # create MLP pipeline for the target and direction
                globals()[f'{direction}_{target}_pipeline_mlp'] = Pipeline([
                    ('scaler', StandardScaler()),
                    ('regressor', MLPRegressor(max_iter=2000, hidden_layer_sizes=(100, 100), verbose=True, n_iter_no_change=25)),
                ])

                # save the pipeline
                pipelines[f'{direction}_{target}_pipeline_mlp'] = globals()[f'{direction}_{target}_pipeline_mlp']

    return pipelines

In [ ]:
def fit_pipelines(pipelines, train_data, target_columns, columns_not_to_use, directions, rf = True, mlp = False):
    # create a dictionary to store the fitted pipelines
    fitted_pipelines = {}

    for target in target_columns:
        target_data = train_data[target]
        
        for direction in directions:

            df = train_data[direction]['X'].copy()
            drop_coumns = columns_not_to_use + target_columns
            df = df.drop(drop_coumns, axis=1)
            
            # get the pipelines
            if rf:
                pipeline_rf = pipelines[f'{direction}_{target}_pipeline_rf']
                
                # fit the pipeline
                pipeline_rf.fit(df, target_data)

                # save the fitted pipeline
                fitted_pipelines[f'{direction}_{target}_pipeline_rf'] = pipeline_rf

                # create a dataframe to store the feature importances
                feature_importances = pd.DataFrame({
                    'feature': df.columns,
                    'importance': pipeline_rf.named_steps['regressor'].feature_importances_
                })

                # sort the features by importance
                feature_importances = feature_importances.sort_values('importance', ascending=False)

                # print the feature importances
                print(f'{direction}_{target}_pipeline_rf')
                print(feature_importances)

            if mlp:
                pipeline_mlp = pipelines[f'{direction}_{target}_pipeline_mlp']
                
                # fit the pipeline
                pipeline_mlp.fit(df, target_data)

                # save the fitted pipeline
                fitted_pipelines[f'{direction}_{target}_pipeline_mlp'] = pipeline_mlp

    return fitted_pipelines

columns_not_to_use = ['datetime', 'dataset', 'day', 'month', 'year', 'casual', 'registered', 'count']
target_columns = ['casual', 'registered', 'count']
directions = ['negative', 'positive']
rf = True
mlp = False

# create the pipelines
pipelines = create_pipelines(target_columns, directions, rf=rf, mlp=mlp)

# fit the pipelines
fitted_pipelines = fit_pipelines(pipelines, training_data, target_columns, directions, rf=rf, mlp=mlp)

In [16]:
# select columns not to be used in the model
columns_not_to_use = ['datetime', 'dataset', 'day', 'month', 'year', 'casual', 'registered', 'count']

# train the models
for target in target_columns:
    for direction in ['positive', 'negative']:
        # train the first pipeline
        globals()[f'{direction}_{target}_pipeline_1'].fit(
            globals()[f'{direction}_train_data'].drop(columns=columns_not_to_use, axis=1),
            globals()[f'{direction}_train_data'][target]
        )

        # save the most important feature into a dataframe
        feature_importances = pd.DataFrame(
            globals()[f'{direction}_{target}_pipeline_1'].named_steps['regressor'].feature_importances_,
            index=globals()[f'{direction}_train_data'].drop(columns=columns_not_to_use, axis=1).columns,
            columns=['importance']
        )

        # sort the dataframe
        feature_importances = feature_importances.sort_values(by='importance', ascending=False)

        # print the most important features
        print(f'{direction} {target} most important features:')
        print(feature_importances)
        print('')

        # train the second pipeline
        globals()[f'{direction}_{target}_mlp_pipeline_2'].fit(
            globals()[f'{direction}_train_data'].drop(columns=columns_not_to_use, axis=1),
            globals()[f'{direction}_train_data'][target]
        )

KeyboardInterrupt: 

In [ ]:
# evaluate the models

for target in target_columns:
    for direction in ['positive', 'negative']:

        # print the statetment of target and direction
        print(f'{direction} {target} evaluation')

        # predict the target using the first pipeline
        globals()[f'{direction}_{target}_predictions_1'] = globals()[f'{direction}_{target}_pipeline_1'].predict(
            globals()[f'{direction}_test_data'].drop(columns=columns_not_to_use, axis=1)
        )

        # calculate the mean absolute error
        globals()[f'{direction}_{target}_mae_1'] = mean_absolute_error(globals()[f'{direction}_test_data'][target], globals()[f'{direction}_{target}_predictions_1'])

        # print the results
        print(f'MAE RF: {globals()[f"{direction}_{target}_mae_1"]}')

        # predict the target using the second pipeline
        globals()[f'{direction}_{target}_predictions_2'] = globals()[f'{direction}_{target}_mlp_pipeline_2'].predict(
            globals()[f'{direction}_test_data'].drop(columns=columns_not_to_use, axis=1)
        )

        # calculate the mean absolute error
        globals()[f'{direction}_{target}_mae_2'] = mean_absolute_error(globals()[f'{direction}_test_data'][target], globals()[f'{direction}_{target}_predictions_2'])

        # print the results
        print(f'MAE MLP: {globals()[f"{direction}_{target}_mae_2"]}')

        # take the average of the two predictions
        globals()[f'{direction}_{target}_predictions'] = (globals()[f'{direction}_{target}_predictions_1'] + globals()[f'{direction}_{target}_predictions_2']) / 2

        # calculate the mean absolute error
        globals()[f'{direction}_{target}_mae'] = mean_absolute_error(globals()[f'{direction}_test_data'][target], globals()[f'{direction}_{target}_predictions'])

        # print the results
        print(f'MAE AVG: {globals()[f"{direction}_{target}_mae"]}')
        print('')

    # combine predictions from both directions
    globals()[f'{target}_predictions'] = (globals()[f'positive_{target}_predictions'] + globals()[f'negative_{target}_predictions']) / 2

    # calculate the mean absolute error
    globals()[f'{target}_mae'] = mean_absolute_error(data[data['dataset'] == 'test'][target], globals()[f'{target}_predictions'])

    # print the results
    print(f'{target} MAE: {globals()[f"{target}_mae"]}')
    print('\n')

NameError: name 'target_columns' is not defined

In [ ]:
# evaluate the models:

for target in target_columns:
    for direction in ['positive', 'negative']:

SyntaxError: incomplete input (3423881943.py, line 4)